#   4.基于概率论的分类方法：朴素贝叶斯
###  优点：在数据较少的情况下仍然有效，可以处理多类别问题。
###    缺点：对于输入数据的准备方式比较敏感。
###    适用数据类型：标称型数据（有限）。
###    注释：“朴素”，这里指整个形式化过程只做最原始、最简单的假设。¶

# 4.1使用Python进行文本分类

## 4.1.1 准备数据：从文本中构建词向量

### 4-1 词表到向量的转化函数

In [1]:
def loadDataSet():
    postingList = [['my','dog','has','flea','problems','help','please'],
                   ['maybe','not','take','him','to','dog','park','stupid'],
                   ['my','dalmation','is','so','cute','I','love','him'],
                   ['stop','posting','stupid','worthless','garbage'],
                   ['mr','licks','ate','my','steak','how','to','stop','him'],
                   ['quit','buying','worthless','dog','food','stupid']]  # 定义邮件列表
    classVec = [0,1,0,1,0,1]  # 1 代表侮辱性文字， 0 代表正常言论
    return postingList, classVec

def createVocabList(dataSet):  # 创建词汇列表
    vocabSet = set([])  # 定义词汇集
    for document in dataSet:  # 遍历文档
        vocabSet = vocabSet | set(document)  # 将每个document合并到vocabSet，|用来联合两个集合
    return list(vocabSet)  
    
def setOfWords2Vec(vocabList, inputSet):  # 把单词转换成向量
    returnVec = [0]*len(vocabList)  # 定义要返回的向量
    for word in inputSet:   # 遍历输出集中的单词
        if word in vocabList:   # 单词在词汇集中
            returnVec[vocabList.index(word)] = 1   # 对应的位置设为1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [2]:
def bagOfWords2VecMN(vocabList, inputSet):   # 把单词转换成向量,用词袋模型，计算词出现的次数
    returnVec = [0]*len(vocabList)   # 定义要返回的向量
    for word in inputSet:  # 遍历输出集中的单词
        if word in vocabList:  # 单词在词汇集中
            returnVec[vocabList.index(word)] += 1    #对应的词出现次数 加1
    return returnVec

## 4.1.2 训练算法：从此向量计算概率
### 4-2 朴素贝叶斯分类器训练函数

In [3]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)   # 计算文档的数目
    numWords = len(trainMatrix[0])   # 计算单词的数目
    pAbusive = sum(trainCategory)/float(numTrainDocs)   # 计算类别的概率，abusive为1，not abusive为0
    p0Num = zeros(numWords)   # 初始化计数器，p0是not abusive
    p1Num = zeros(numWords) # 初始化计数器
    p0Denom = 0.0  # 初始化分母
    p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:   # 计算abusive对应的词汇的数目，trainMatrix为0-1值形成的向量
            p1Num += trainMatrix[i]    # p1Num存储的是每个词出现的次数
            p1Denom += sum(trainMatrix[i])   # p1Denom存储的是词的总数目
        else:
            p0Num += trainMatrix[i]   # 每个词在not abusive下出现的次数
            p0Denom += sum(trainMatrix[i])   # not abusive下的总词数
    p1Vect = p1Num/p1Denom   # change to log()   # 计算abusive下每个词出现的概率
    p0Vect = p0Num/p0Denom   # change to log()   # 计算not abusive下每个词出现的概率
    return p0Vect,p1Vect,pAbusive

## 4.1.3 测试算法：根据现实情况修改分类器
### 4-3 朴素贝叶斯分类函数

In [4]:
from numpy import * 
def classifyNB(vec2Classify, p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)  # log()默认以e为底     ### 计算abusive的概率
    p0 = sum(vec2Classify * p0Vec) + log(1.0-pClass1)  # 计算not abusive的概率
    if p1 > p0:
        return 1
    else:
        return 0
    
    
def testingNB():
    listOPosts, listClasses = loadDataSet()   
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,"classified as:",classifyNB(thisDoc,p0V,p1V,pAb))
    
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,"classified as:",classifyNB(thisDoc,p0V,p1V,pAb))

In [5]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


# 4.2 示例：使用朴素贝叶斯过滤垃圾邮件

## 4.2.1 准备数据：切分文本

In [6]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent1 = mySent.split()  #  问题： 标点符号也被当成此的一部分
print(mySent1)

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M.L.', 'I', 'have', 'ever', 'laid', 'eyes', 'upon.']


In [7]:
# 解决标点符号也被当成此的一部分的问题  ———— 使用正则表示式，分隔符是除了 |单词， 数字| 外的任意字符串
import re
regEx = re.compile('\\W*')
listOfToken = regEx.split(mySent)
print(listOfToken)
a = [tok for tok in listOfToken if len(tok) > 0]  # 除去 空字符
print(a)
b = [tok.lower() for tok in listOfToken if len(tok) > 0]
print(b)

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon', '']
['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon']
['this', 'book', 'is', 'the', 'best', 'book', 'on', 'python', 'or', 'm', 'l', 'i', 'have', 'ever', 'laid', 'eyes', 'upon']


c:\users\dell\appdata\local\juliapro-0.6.4.1\python\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: split() requires a non-empty pattern match.
  after removing the cwd from sys.path.


## 4.2.2 测试算法：使用朴素贝叶斯进行交叉验证
### 4-5 文件解析以及完整的垃圾邮件测试函数

In [8]:
def textParse(bigString):  # 文本解析   ### 输入是字符串，输出是单词列表
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList = []  # 定义docList文档列表
    classList = []  # classList类别列表
    fullText = []   # fullText所有文档词汇
    for i in range(1,26):
        # 定义并读取垃圾邮件文件的词汇分割列表
        wordList = textParse(open('email/spam/%d.txt' % i).read())   # !!! 为了避免文件中非法字符的影响，尽量用下一行
        #wordList = textParse(open('email/spam/%d.txt' % i, "rb").read().decode('GBK','ignore'))
        
        docList.append(wordList)   # 将词汇列表加到文档列表中
        fullText.extend(wordList)   # 将所有词汇列表汇总到fullText中
        classList.append(1)   # 文档类别为1，spam
        
        #wordList = textParse(open('email/ham/%d.txt' % i).read())   # !!! 原书本中 bug 因为有可能文件中存在类似“�”非法字符。
        # 提示：UnicodeDecodeError: 'gbk' codec can't decode byte 0xae in position 199: illegal multibyte sequence
        wordList = textParse(open('email/ham/%d.txt' % i,  "rb").read().decode('GBK','ignore') )  # 读取非垃圾邮件的文档
        
        docList.append(wordList)  
        fullText.extend(wordList)
        classList.append(0)  # 类别为0，非垃圾邮件
        
    vocabList = createVocabList(docList)
    #trainingSet = range(50)  # !!! 原书本中 python2.7   提示：TypeError: 'range' object doesn't support item deletion
    #因为是python3中range不返回数组对象，而是返回range对象 
    trainingSet = list(range(50))   # 定义训练集的索引和测试集
    testSet = []
    for i in range(10):   # 随机的选择10个作为测试集
        randIndex = int(random.uniform(0,len(trainingSet)))  # 随机索引
        testSet.append(trainingSet[randIndex])  # 将随机选择的文档加入到测试集中
        del(trainingSet[randIndex])  # 从训练集中删除随机选择的文档
    trainMat = []   # 定义训练集的矩阵和类别 
    trainClasses = []
    for docIndex in trainingSet:    # 遍历训练集，求得先验概率和条件概率
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))   # 将词汇列表变为向量放到trainMat中
        trainClasses.append(classList[randIndex])   # 训练集的类别标签
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))    # 计算先验概率，条件概率
    errorCount = 0    
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])   # 将测试集词汇向量化
        if classifyNB(array(wordVector), p0V, p1V,pSpam) != classList[docIndex]:   # 对测试数据进行分类
            errorCount += 1   # 分类不正确，错误计数加1
    print("the error rate is:",float(errorCount)/len(testSet))   

In [9]:
spamTest()

the error rate is: 0.3


c:\users\dell\appdata\local\juliapro-0.6.4.1\python\python35.zip\re.py:203: FutureWarning: split() requires a non-empty pattern match.
c:\users\dell\appdata\local\juliapro-0.6.4.1\python\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
c:\users\dell\appdata\local\juliapro-0.6.4.1\python\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


# 4.3 示例：使用朴素贝叶斯分类器从个人广告中获取区域倾向
### （1）收集数据：从RSS源收集内容，这里需要对RSS源构建一个接口。
### （2）准备数据：将文本文件解析成词条向量
### （3）分析数据：检查词条确保解析的正确性。
### （4）训练算法：使用之前建立的trainNB0（）函数。
### （5）测试算法：观察错误率，确保分类器可用。可以修改切分程序，以降低错误率，提高分类效果。
### （6）使用算法：构建一个完整的程序，封装所有内容。给定两个RSS源，该程序会显示最常用的公共词。
## 4.3.1 收集数据：导入RSS源
### 4-6 RSS源分类器集高频词去除函数

In [12]:
def calcMostFreq(vocabList,fullText):  
    import operator 
    freqDict = {}
    for token in vocabList:  # 遍历词汇表中每个单词
        freqDict[token] = fullText.count(token)   # 统计词在文本中出现的次数
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1),reverse=True)  # 根据出现的次数从高到低对词典进行排序
    return sortedFreq[:30]   # 返回排序最高的30个单词

def localWords(feed1,feed0):
    import feedparser  
    docList = []   # 定义docList文档列表
    classList = []   # classList类别列表
    fullText = []   # fullText所有文档词汇列表
    minLen = min(len(feed1['entries']),len(feed0['entries']))   # miNLen记录帖子数少的数目
    
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])  #取出内容，并拆成词
        docList.append(wordList)   # wordList单词列表以（子列表）的形式  添加到 docList文档列表 
        fullText.extend(wordList)  # wordList单词列表以（普通元素——非列表形式） 扩展到 fullText列表
        classList.append(1)  # ny的标签是1
        wordList = textParse(feed0['entries'][i]['summary'])  # 同上
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) # sf的标签是0
    vocabList = createVocabList(docList)  #创建词汇表
    top30Words = calcMostFreq(vocabList,fullText)  # 从fulltext中找出最高频的30个单词,并从vocabList中去除它们
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    trainingSet = list(range(2*minLen))   #创建训练集
    testSet = []   # 测试集
    
    for i in range(20):    #随机选取20的数据，建立测试集
        randIndex = int(random.uniform(0,len(trainingSet)))   # 在范围 0 ~ 训练集长度 内产生一个随机数，并转化为int类型
        testSet.append(trainingSet[randIndex])    # 在测试集列表 增加 训练集的随机元素 
        del(trainingSet[randIndex])  # 删除出现过的随机元素，避免重复
    trainMat = []
    trainClasses = []
    
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))    #将训练集中的每一条数据，转化为词向量
        trainClasses.append([docIndex])     
    
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))   #开始训练
    errorCount = 0
    
    for docIndex in testSet:   # 用测试数据，测试分类器的准确性
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])   # 把单词转换成向量
        if classifyNB(array(wordVector), p0V,p1V,pSpam) != classList[docIndex]:  # 如果分类错误
            errorCount += 1
    print("the error rate is:",float(errorCount)/len(testSet))  # 输出错误率
    return vocabList,p0V,p1V

In [13]:
import feedparser
ny = feedparser.parse('http://www.capitalpress.com/section/rssfeed/1174&template=rss&mime=xml')
sf = feedparser.parse('http://www.capitalpress.com/section/rssfeed/1178&template=rss&mime=xml')
res = len(ny['entries'])
res1 = len(sf['entries'])
print(res, res1)
vocabList,pSF,pNY = localWords(ny,sf)

print(vocabList,pSF,pNY)

25 25
the error rate is: 1.0
['take', 'cover', 'washington', 'managers', 'top', 'grower', 'fighting', 'hundreds', 'around', 'predators', 'were', 'conservation', 'lowest', 'fall', 'zinke', 'along', 'lead', 'making', 'took', 'crowded', 'forcing', 'arizona', 'recommended', '2017', 'uses', 'lifted', 'protect', 'megafires', 'bondurant', 'controlling', 'estimated', 'down', '1970s', 'hunting', 'picked', 'montana', 'over', 'would', 'store', 'efforts', 'just', 'additional', 'runoff', 'percent', 'years', 'modoc', 'accidentally', 'critics', 'bear', 'hotter', 'existing', 'pounds', 'animals', 'one', 'utah', 'supposed', 'out', 'fuel', 'cities', 'behind', 'southwestern', 'agency', 'oil', 'become', 'sunday', 'extremely', 'royalties', 'horses', 'funding', 'moving', 'some', 'examiners', 'there', 'recreation', 'farmers', 'away', 'high', 'weighing', 'buck', 'mountain', 'elsewhere', 'early', 'southern', 'almost', 'united', 'wednesday', 'cost', 'subject', 'elk', 'expansion', 'should', 'asking', 'aims', 'smo

c:\users\dell\appdata\local\juliapro-0.6.4.1\python\python35.zip\re.py:203: FutureWarning: split() requires a non-empty pattern match.
c:\users\dell\appdata\local\juliapro-0.6.4.1\python\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
